In [1]:
import numpy as np
import pandas as pd

import os, shutil, importlib
import utils; importlib.reload(utils)

from glob import glob

%matplotlib inline
np.set_printoptions(precision=4, linewidth=100)

In [3]:
path = '../data/dogscats/'
classes = ['cat', 'dog']

utils.struct_dir(path, classes)

Moving cat: 11500 to train, 1000 to valid
Moving dog: 11500 to train, 1000 to valid
